In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crypto-regulations-yirifi/mappingtoBERT_6.csv
/kaggle/input/crypto-regulations-yirifi/mappingtoBERT_5.csv
/kaggle/input/crypto-regulations-yirifi/output_improved.csv
/kaggle/input/crypto-regulations-yirifi/master_data_gpt4o_improved_latest.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/18.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/20.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/1.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/24.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/11.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/30.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/17.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/16.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/3.csv
/kaggle/input/crypto-regulations-

# Data import and cleaning up the data

In [2]:
df = pd.read_csv("/kaggle/input/crypto-regulations-yirifi/mappingtoBERT_6.csv")

In [3]:
df.shape

(5965, 4)

In [4]:
df.tail(10)

,Unnamed: 0,Content,Concepts,bert_concepts
5955,5955,Content|ÄúRegulationsÄù|means the Virtual As...,"amended, related activities, 2023, supplemente...","regulations, virtual assets"
5956,5956,Content|ÄúReserve AssetsÄù|has the meaning a...,"frva rules, reserve assets, rule i.a.5",rules
5957,5957,Content|ÄúSenior ManagementÄù|has the meanin...,"company rulebook, senior management",rulebook
5958,5958,Content|ÄúSignificant FRVA IssuerÄù|means a ...,"rule i.c.1, vara, frva rules, vasp, significan...","rules, vasp"
5959,5959,Content|ÄúTechnology and Information\nRuleboo...,"amended, information, vara, technology, regula...","regulations, rulebook"
5960,5960,Content|ÄúVA Issuance RulebookÄù|means the V...,"amended, supplemented, vara, virtual asset iss...","regulations, virtual assets, rulebook"
5961,5961,Content|ÄúVARAÄù|means the Dubai Virtual Ass...,"vara, dubai, regulatory authority, virtual assets","authority, virtual assets, regulator"
5962,5962,Content|ÄúVirtual AssetÄù or ÄúVAÄù|has th...,"dubai va law, va, regulatory, compliance, virt...","compliance, virtual assets, regulator"
5963,5963,Content|ÄúWhitepaperÄù|has the meaning ascri...,"bank, whitepaper, regulatory compliance, compl...","compliance, virtual assets, rulebook, regulator"
5964,5964,Content|ÄúWorking DayÄù|means any day which ...,"public holiday, weekend, bank, hong kong, regu...","compliance, virtual assets, regulator"


In [5]:
import re
# Function to check for URLs
def contains_url(text):
    text = str(text)
    return bool(re.search(r'http\S+|www\S+|https\S+', text))

# Function to check for user references (@user)
def contains_user_reference(text):
    text = str(text)
    return bool(re.search(r'\@\w+', text))

# Function to check for hashtags
def contains_hashtag(text):
    text = str(text)
    return bool(re.search(r'\#\w+', text))

# Function to check for non-alphanumeric characters
def contains_non_alphanumeric(text):
    text = str(text)
    return bool(re.search(r'[^a-zA-Z0-9\s]', text))

# Apply the functions to the DataFrame
df['contains_url'] = df['Content'].apply(contains_url)
df['contains_user_reference'] = df['Content'].apply(contains_user_reference)
df['contains_hashtag'] = df['Content'].apply(contains_hashtag)
df['contains_non_alphanumeric'] = df['Content'].apply(contains_non_alphanumeric)

# Check rows that match each condition
url_count = df['contains_url'].sum()
user_reference_count = df['contains_user_reference'].sum()
hashtag_count = df['contains_hashtag'].sum()
non_alphanumeric_count = df['contains_non_alphanumeric'].sum()

# Print results
print(f"Number of rows with URLs: {url_count}")
print(f"Number of rows with user references: {user_reference_count}")
print(f"Number of rows with hashtags: {hashtag_count}")
print(f"Number of rows with non-alphanumeric characters: {non_alphanumeric_count}")



Number of rows with URLs: 22
Number of rows with user references: 12
Number of rows with hashtags: 0
Number of rows with non-alphanumeric characters: 5913


In [6]:
df.tail(5)

,Unnamed: 0,Content,Concepts,bert_concepts,contains_url,contains_user_reference,contains_hashtag,contains_non_alphanumeric
5960,5960,Content|ÄúVA Issuance RulebookÄù|means the V...,"amended, supplemented, vara, virtual asset iss...","regulations, virtual assets, rulebook",False,False,False,True
5961,5961,Content|ÄúVARAÄù|means the Dubai Virtual Ass...,"vara, dubai, regulatory authority, virtual assets","authority, virtual assets, regulator",False,False,False,True
5962,5962,Content|ÄúVirtual AssetÄù or ÄúVAÄù|has th...,"dubai va law, va, regulatory, compliance, virt...","compliance, virtual assets, regulator",False,False,False,True
5963,5963,Content|ÄúWhitepaperÄù|has the meaning ascri...,"bank, whitepaper, regulatory compliance, compl...","compliance, virtual assets, rulebook, regulator",False,False,False,True
5964,5964,Content|ÄúWorking DayÄù|means any day which ...,"public holiday, weekend, bank, hong kong, regu...","compliance, virtual assets, regulator",False,False,False,True


In [7]:
df = df[["Content","bert_concepts"]]

In [8]:
import re
import string

def preprocess_text(text):
    text = str(text)    
    text = re.sub(r'<[^>]+>', '', text) # Remove HTML tags
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)# Remove URLs
    text = re.sub(r'\@\w+|\#','', text) # Remove user @ references and hashtags
    text = text.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation    
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text) # Remove non-alphanumeric character    
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace(only single white space is left)
    
    return text

df['Content'] = df['Content'].apply(preprocess_text) # Applying the preprocess_text function


In [9]:
df.tail(5)

,Content,bert_concepts
5960,content va issuance rulebook means the virtual...,"regulations, virtual assets, rulebook"
5961,content vara means the dubai virtual assets re...,"authority, virtual assets, regulator"
5962,content virtual asset or va has the meaning as...,"compliance, virtual assets, regulator"
5963,content whitepaper has the meaning ascribed to...,"compliance, virtual assets, rulebook, regulator"
5964,content working day means any day which is not...,"compliance, virtual assets, regulator"


In [10]:
df.shape

(5965, 2)

In [11]:
missing_values_count = df['bert_concepts'].isnull().sum()
print("Number of rows with NaN in bert_concepts:", missing_values_count)

Number of rows with NaN in bert_concepts: 580


## Remove rows with no bert_concepts

In [12]:
df = df.dropna(subset=['bert_concepts'])

In [13]:
df.shape

(5385, 2)

# Converting to Bert Data Format

In [14]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

In [15]:
df.columns

Index(['Content', 'bert_concepts'], dtype='object')

In [16]:

df['bert_concepts'] = df['bert_concepts'].apply(lambda x: x.split(', ')) # make list of words
all_concepts = set(concept for row in df['bert_concepts'] for concept in row) # Create a list of all unique concepts


In [17]:
all_concepts

{'aml',
 'anti-bribery',
 'audit',
 'authority',
 'beneficiary information',
 'cdd',
 'cft',
 'circular',
 'client',
 'code of conduct',
 'compliance',
 'conduct',
 'controls',
 'criminal',
 'disclosure',
 'dlt',
 'due diligence',
 'entities',
 'guideline',
 'identity',
 'insider dealing',
 'investor',
 'investor protection',
 'kyc',
 'legal',
 'legislation',
 'license',
 'monitoring',
 'operator',
 'oversight',
 'pep',
 'person',
 'policies',
 'policy',
 'record keeping',
 'regulations',
 'regulator',
 'reporting',
 'requirements',
 'risk assessment',
 'risk management',
 'risks',
 'rulebook',
 'rules',
 'safeguard',
 'sanctions',
 'screening',
 'suspicious transactions',
 'transaction',
 'transactions',
 'transfer',
 'travel rule',
 'vasp',
 'vasps',
 'virtual assets'}

In [18]:
# One-hot encode each concept for each row
one_hot_encoded_rows = []
for index, row in df.iterrows():
    one_hot_row = {'Content': row['Content']}
    for concept in all_concepts:
        one_hot_row[concept] = True if concept in row['bert_concepts'] else False
    one_hot_encoded_rows.append(one_hot_row)


In [19]:
one_hot_encoded_rows[:2]

[{'Content': 'the new licensing regime for centralised virtual asset trading platforms under the antimoney laundering and counterterrorist financing ordinance cap 615 amlo will come into effect on 1 june 2023 under the new regime centralised virtual asset trading platforms operating in hong kong will need to apply to the securities and futures commission sfc for a licence under the securities and futures ordinance cap 571 sfo andor the amlo dual licence arrangement',
  'transfer': False,
  'policy': False,
  'legislation': False,
  'criminal': False,
  'vasps': False,
  'travel rule': False,
  'transaction': False,
  'transactions': False,
  'rulebook': False,
  'controls': False,
  'insider dealing': False,
  'dlt': False,
  'requirements': False,
  'guideline': False,
  'virtual assets': True,
  'investor protection': False,
  'risk management': False,
  'investor': False,
  'rules': False,
  'client': False,
  'regulator': True,
  'audit': False,
  'vasp': False,
  'screening': Fals

In [20]:
df = pd.DataFrame(one_hot_encoded_rows)

In [21]:
df.head(3)

,Content,transfer,policy,legislation,criminal,vasps,travel rule,transaction,transactions,rulebook,...,suspicious transactions,cdd,aml,cft,beneficiary information,identity,operator,legal,circular,oversight
0,the new licensing regime for centralised virtu...,False,False,False,False,False,False,False,False,False,...,False,False,True,True,False,False,False,False,False,False
1,the sfc is issuing this circular to highlight ...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,the sfc has published the following guidelines...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [22]:
import pandas as pd
from datasets import Dataset

In [23]:
#Splitting DataFrame into train, test, and validation sets
train_df, test_valid_df = train_test_split(df, test_size=0.3, random_state=42)
test_df, validation_df = train_test_split(test_valid_df, test_size=0.33, random_state=42)

# Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
validation_dataset = Dataset.from_pandas(validation_df)

# Create DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "validation": validation_dataset
})




In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Content', 'transfer', 'policy', 'legislation', 'criminal', 'vasps', 'travel rule', 'transaction', 'transactions', 'rulebook', 'controls', 'insider dealing', 'dlt', 'requirements', 'guideline', 'virtual assets', 'investor protection', 'risk management', 'investor', 'rules', 'client', 'regulator', 'audit', 'vasp', 'screening', 'record keeping', 'code of conduct', 'person', 'regulations', 'license', 'disclosure', 'policies', 'risk assessment', 'compliance', 'reporting', 'conduct', 'entities', 'monitoring', 'pep', 'safeguard', 'anti-bribery', 'authority', 'sanctions', 'due diligence', 'kyc', 'risks', 'suspicious transactions', 'cdd', 'aml', 'cft', 'beneficiary information', 'identity', 'operator', 'legal', 'circular', 'oversight', '__index_level_0__'],
        num_rows: 3769
    })
    test: Dataset({
        features: ['Content', 'transfer', 'policy', 'legislation', 'criminal', 'vasps', 'travel rule', 'transaction', 'transactions', 'r

In [25]:
[i for i in dataset['train']][:2]

[{'Content': 'a shall not by itself render that person liable to any judicial or other proceedings but in any proceedings under the sfo andor the amlo before any court these guidelines shall be admissible in evidence and if any provision set out in these guidelines appears to the court to be relevant to any question arising in the proceedings it shall be taken into account in determining the question and',
  'transfer': False,
  'policy': False,
  'legislation': False,
  'criminal': False,
  'vasps': False,
  'travel rule': False,
  'transaction': False,
  'transactions': False,
  'rulebook': False,
  'controls': False,
  'insider dealing': False,
  'dlt': False,
  'requirements': False,
  'guideline': True,
  'virtual assets': False,
  'investor protection': False,
  'risk management': False,
  'investor': False,
  'rules': False,
  'client': False,
  'regulator': False,
  'audit': False,
  'vasp': False,
  'screening': False,
  'record keeping': False,
  'code of conduct': False,
  '

# Data Ready Now Training

In [26]:
!pip install -q transformers datasets

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info/METADATA'



Add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.

### List containing all the labels

In [27]:
labels = [label for label in dataset['train'].features.keys() if label not in ['Content']]
#Create 2 dictionaries that map labels to integers and back.
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['transfer',
 'policy',
 'legislation',
 'criminal',
 'vasps',
 'travel rule',
 'transaction',
 'transactions',
 'rulebook',
 'controls',
 'insider dealing',
 'dlt',
 'requirements',
 'guideline',
 'virtual assets',
 'investor protection',
 'risk management',
 'investor',
 'rules',
 'client',
 'regulator',
 'audit',
 'vasp',
 'screening',
 'record keeping',
 'code of conduct',
 'person',
 'regulations',
 'license',
 'disclosure',
 'policies',
 'risk assessment',
 'compliance',
 'reporting',
 'conduct',
 'entities',
 'monitoring',
 'pep',
 'safeguard',
 'anti-bribery',
 'authority',
 'sanctions',
 'due diligence',
 'kyc',
 'risks',
 'suspicious transactions',
 'cdd',
 'aml',
 'cft',
 'beneficiary information',
 'identity',
 'operator',
 'legal',
 'circular',
 'oversight',
 '__index_level_0__']